### Request API notebook

The initial step of the project consists of requesting flight information from [Aviation-Edge API](https://aviation-edge.com).  This website allows you to request data from airplanes with the status of "landed" which means that the airplane has take-off and arrived at the destination airport. Since this project aims to analyze flights from Europe only, especially from Barcelona airport, a second request was made to gather information about all the airports in Spain, Italy, Germany, and France. This second step is to obtain the airports IATA codes to use it as a parameter in the request flight information function. 

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import pymysql
from sqlalchemy import create_engine
import re
import requests as req
import json
import datetime
import time

In [2]:
KEY = 'xxxxxxxxxxxx'

BCN = 'BCN'

status = 'landed'

url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={BCN}&type=departure&status={status}"


## Request flight information function.

This function request data from the API using a key or token, the IATA code of the airport of interest and the status which in this case is "landed" as mentioned before. 

In [3]:
def get_data(airport,status = 'landed'):
    
    KEY = 'xxxxxxxxxxxx'
    url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={airport}&type=departure&status={status}"    
    response = req.get(url)
    print(response)
    print((datetime.datetime.now()))
    json_file = response.json()
    #Converting the json file info a Pandas data frame. 
    if type(json_file) == list:
        df_f = pd.DataFrame(json_file)

        return df_f
    
    

## Cleaning the data frame. 

When converting the JSON file into a data frame, the names and values of the columns were in in the shape of a dictionary. Based on this, I had to clean them and only get the information of interest. 

For example, in the key "airline" and wanted to get the "name" and the "iataCode" values. In order to perform this task, a lambda function was applied as shown in the function below. 

In [4]:
def clean_data(data_df):
    data_df["airln_name"] = data_df["airline"].apply(lambda x: x['name'])
    data_df["airln_code"] = data_df["airline"].apply(lambda x: x['iataCode'])
    data_df['arri_delay'] = data_df['arrival'].apply(lambda x: x['delay'])
    data_df['arri_airpt'] = data_df['arrival'].apply(lambda x: x['iataCode'])
    data_df['schd_arri_time'] = data_df['arrival'].apply(lambda x: x['scheduledTime'])
    data_df['codeshared'] = data_df['codeshared'].apply(lambda x: x['airline']['name'] if x else None)
    data_df['depart_delay'] = data_df['departure'].apply(lambda x: x['delay'])
    data_df['depart_airpt'] = data_df['departure'].apply(lambda x: x['iataCode'])
    data_df['schd_depart_time'] = data_df['departure'].apply(lambda x: x['scheduledTime'])
    data_df['depart_gate'] = data_df['departure'].apply(lambda x: x['gate'])
    data_df['depart_terminal'] = data_df['departure'].apply(lambda x: x['terminal'])
    data_df['icaonumb'] = data_df['flight'].apply(lambda x: x['icaoNumber'])
    data_df['flight'] = data_df['flight'].apply(lambda x: x['number'])
    data_df = data_df.drop(['airline','arrival','departure'], axis =1)
    
    return data_df
    

## Get airports by country

The function below request data from the airports on interest. In this case, the returning data frame does not need to be clean. Moreover, as I only needed the name, IATA code and ISO code from the data frame, a second function was developed.

As a final step, a new data frame was created and loaded to the database schema in SQL PRO. 

In [6]:
def get_data_airpts(country):
    
    KEY = 'b50bc5-5af08a'
    url_airpt = f'https://aviation-edge.com/v2/public/airportDatabase?key={KEY}&codeIso2Country={country}'
    
    response = req.get(url_airpt)
    print(response)
    print((datetime.datetime.now()))
    json_file = response.json()
    df = pd.DataFrame(json_file)
    return df

In [7]:
spain_airpts = get_data_airpts('ES')

<Response [200]>
2019-07-29 09:15:02.353142


In [8]:
spain_airpts.head(2)

,GMT,airportId,codeIataAirport,codeIataCity,codeIcaoAirport,codeIso2Country,geonameId,latitudeAirport,longitudeAirport,nameAirport,nameCountry,phone,timezone
0,1,27,ABC,ABC,LEAB,ES,6299323,38.948334,-1.863333,Los Llanos,Spain,,Europe/Madrid
1,0,55,ACE,ACE,GCRR,ES,6297189,28.950668,-13.609059,Lanzarote,Spain,902 404 704,Atlantic/Canary


In [ ]:
# A function to get only the IATA codes from the dataframe from above

In [9]:
def get_iata_codes(df_airpts):
    airpts_lst = df_airpts['codeIataAirport'].to_list()
    country_code = df_airpts['codeIso2Country'].to_list()
    name = df_airpts['nameAirport'].to_list()
    return airpts_lst,country_code, name

## Requesting information from the countries of interest.

### Spain

In [10]:
spain_airpts_lst, country_code_sp, name_airport = get_iata_codes(df_airpts=spain_airpts)


In [11]:
print(spain_airpts_lst)

['ABC', 'ACE', 'AEI', 'AGP', 'ALC', 'BCN', 'BIO', 'BJZ', 'CDT', 'CDZ', 'EAS', 'EEL', 'EEM', 'EER', 'EEU', 'FES', 'FGR', 'FUE', 'GEN', 'GGA', 'GMZ', 'GRO', 'GRX', 'HEV', 'HSK', 'IBZ', 'ILD', 'JCU', 'LCG', 'LEI', 'LEN', 'LEU', 'LPA', 'MAD', 'MAH', 'MJV', 'MLN', 'NDO', 'ODB', 'OVD', 'OZI', 'OZL', 'OZP', 'OZU', 'PMI', 'PNA', 'PXS', 'QGN', 'QLQ', 'REU', 'RGS', 'RJL', 'RMU', 'ROZ', 'RRA', 'SCQ', 'SDR', 'SLM', 'SPC', 'SPO', 'SVQ', 'TFN', 'TFS', 'TOJ', 'UER', 'UTL', 'VDE', 'VGO', 'VIT', 'VLC', 'VLL', 'XIV', 'XJB', 'XJI', 'XJJ', 'XJN', 'XJO', 'XJR', 'XJU', 'XOC', 'XOJ', 'XOU', 'XQA', 'XRY', 'XTI', 'XTJ', 'XUA', 'XUF', 'XUT', 'XZZ', 'YJB', 'YJC', 'YJD', 'YJE', 'YJG', 'YJH', 'YJI', 'YJL', 'YJM', 'YJR', 'YJV', 'YJW', 'ZAZ', 'ZRC']


In [13]:
spain_airports = pd.DataFrame({'country_code':country_code_sp,
             'iata_code':spain_airpts_lst,
                              'name': name_airport})

In [13]:
spain_airports.to_sql('airports',engine, if_exists='append', index=False)

### France

In [12]:
france_airpts = get_data_airpts('FR')

<Response [200]>
2019-07-29 09:15:24.899679


In [13]:
france_airpts_lst,country_code_fr, name_airport = get_iata_codes(df_airpts=france_airpts)

In [16]:
france_airports = pd.DataFrame({'country_code':country_code_fr,
             'iata_code':france_airpts_lst,
                              'name': name_airport})

In [17]:
france_airports.to_sql('airports',engine, if_exists='append', index=False)

### Germany (DE)

In [14]:
germany_airpts = get_data_airpts('DE')

<Response [200]>
2019-07-29 09:15:33.430408


In [15]:
germany_airpts_lst,country_code_de, name_airport = get_iata_codes(df_airpts=germany_airpts)

In [19]:
germany_airports = pd.DataFrame({'country_code':country_code_de,
             'iata_code':germany_airpts_lst,
                              'name': name_airport})

In [21]:
germany_airports.to_sql('airports',engine, if_exists='append', index=False)

### Italy

In [16]:
italy_airpts = get_data_airpts('IT')

<Response [200]>
2019-07-29 09:15:37.816071


In [17]:
italy_airpts_lst, country_code_it, name_airport = get_iata_codes(df_airpts=italy_airpts)

In [22]:
italy_airports = pd.DataFrame({'country_code':country_code_it,
             'iata_code':italy_airpts_lst,
                              'name': name_airport})

In [25]:
italy_airports.to_sql('airports',engine, if_exists='append', index=False)

## All countries airports iata codes

In [18]:
countries_airpts_lst = spain_airpts_lst + france_airpts_lst + germany_airpts_lst + italy_airpts_lst

In [19]:
df_countries_airpt = pd.DataFrame(countries_airpts_lst)

# Streaming the data from the API

This function calls the "get_data" and "clean_data" functions described at the beginning using the airports IATA codes gathered trough the "get_iata_codes function". 

In [ ]:
dict_airpt_df = {}

for aipt_code in countries_airpts_lst:
    
    df_airpt = get_data(airport = f"{aipt_code}")
    
    if type(df_airpt) == pd.core.frame.DataFrame:
        df_airpt_clean = clean_data(df_airpt)
        dict_airpt_df[aipt_code] = df_airpt_clean
        time.sleep(2)


## Saving and cleaning the data to SQL Pro. 

Finally, a new function was created to clean the data in terms of data types, null values, and dropping duplicate values and uploading the final results to SQL PRO. 

In [40]:
from sqlalchemy import create_engine


driver = 'mysql+pymysql:'
user = 'root'
password = 'xxxxxxxxxxxx'
ip = '127.0.0.1'
database = 'project_4'

In [41]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)


In [42]:
def save_to_sql_clean():
    for key, value in dict_airpt_df.items():
        
        value = value.astype({'schd_arri_time': 'datetime64[ns]','schd_depart_time':'datetime64[ns]'})
        value = value[['airln_name', 'airln_code','type', 'codeshared', 'status', 'icaonumb', 'flight','depart_terminal',\
      'depart_gate', 'depart_airpt' ,'schd_depart_time','depart_delay','arri_airpt','schd_arri_time',\
      'arri_delay']]
        value['depart_delay'] = value['depart_delay'].fillna(0)
        value['arri_delay'] = value['arri_delay'].fillna(0)
        value = value.astype({'depart_delay': int,'arri_delay':int})
        value = value.drop_duplicates(subset = ['flight','depart_terminal','schd_depart_time', 'depart_delay', \
                              'schd_arri_time', 'arri_delay'], keep='first')
        
        value.to_sql('flights',engine, if_exists='append', index=False)
        
        
        
        

In [43]:
save_to_sql_clean()